In [0]:
import os
os.listdir("/")

In [0]:
display(dbutils.fs.ls("/nuevomodelo/modelrnn_nuevo2"))

In [0]:
import tensorflow as tf
# Paso 1: Copiar el modelo desde el almacenamiento Azure Storage al Directorio LOCAL(.OS) -- Obligatorio Poner file:
dbutils.fs.cp("/mnt/datalakemlopsd4m/presentation/ModelsResults/modelornn_version1", "file:/nuevomodelo/modelrnn_nuevo3", recurse=True)

In [0]:
# Paso 2: Cargar el Modelo ML desde el Directorio LOCAL(.OS) -- No es necesario poner file:
model2 = tf.keras.models.load_model("/nuevomodelo/modelrnn_nuevo3")

In [0]:
#Paso 3: Hace la prediccion con el modelo cargado del Azure Storage
model2.predict([[12,45,229,4,129]])

In [0]:
import mlflow
import mlflow.keras

# Especifica el nombre del modelo registrado en MLflow Registry
model_name = "MLops_kerastf_v1"

# Especifica la versión del modelo que deseas cargar
model_version = "1"

# Carga el modelo desde MLflow Registry especificando la versión
keras_model = mlflow.keras.load_model(f"models:/{model_name}/{model_version}")

In [0]:
import numpy as np
# Datos de entrada
input_data = np.array([[12, 35, 129, 4, 129]])

# Remodelar los datos de entrada para que tengan la forma (None, 1, 5)
input_data_reshaped = np.reshape(input_data, (input_data.shape[0], 1, input_data.shape[1]))


keras_pred = keras_model.predict(input_data_reshaped)
keras_pred

In [0]:
from mlflow.tracking import MlflowClient
endpoint_name = "mlops_kerasv1_endpoint"
model_name = "MLops_kerastf_v1"

#ELEGIR UNA VERSION EN ESPECIFICO
#specific_model_version = 1  # Especifica la versión deseada del modelo
#Accede a la versión específica del modelo
#model_version = MlflowClient().get_model_version(model_name, specific_model_version).version

#ELEGIR LA ULTIMA VERSION DISPONIBLE
model_version = MlflowClient().get_registered_model(model_name).latest_versions[0].version 
workload_type = "GPU_LARGE"  #tipo de carga de trabajo que se ejecutará en el endpoint de producción. Puede ser CPU, GPU_SMALL, GPU_LARGE
workload_size = "Small"  # indica el tamaño del recurso de cómputo que se asignará al endpoint de producción. Puede ser Small, Medium, Large
scale_to_zero = False  # Si se establece en True, el sistema puede apagar automáticamente el endpoint cuando no esté recibiendo solicitudes

In [0]:
import requests
import json

#apiUrl de la API que se utilizará para interactuar con el servicio de MLflow
API_ROOT = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get() 

#apiToken de autorización necesario para autenticar las solicitudes a la API
API_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()  

#Cabeceras de la solicitud HTTP
data = {
    "name": endpoint_name,
    "config": {
        "served_entities": [
            {
                "entity_name": model_name,
                "entity_version": model_version,
                "workload_size": workload_size,
                "scale_to_zero_enabled": scale_to_zero,
                "workload_type": workload_type,
            }
        ]
    },
}

# Token de autorización 
headers = {"Context-Type": "text/json", "Authorization": f"Bearer {API_TOKEN}"}

#Se realiza una solicitud POST a la URL de la API de MLflow para crear el endpoint de producción
response = requests.post(
    url=f"{API_ROOT}/api/2.0/serving-endpoints", json=data, headers=headers
)

print(json.dumps(response.json(), indent=4))

In [0]:
# send the POST request to create the serving endpoint

# Datos de entrada
input_data = np.array([[12, 35, 129, 4, 129]])

# Remodelar los datos de entrada para que tengan la forma (None, 1, 5)
input_data_reshaped = np.reshape(input_data, (input_data.shape[0], 1, input_data.shape[1]))

# Convertir los datos de entrada a una lista de Python para serializarlos a JSON
input_data_list = input_data_reshaped.tolist()

data = {
  "inputs" : input_data_list,
  "params" : {"max_new_tokens": 100, "temperature": 1}
}

headers = {"Context-Type": "text/json", "Authorization": f"Bearer {API_TOKEN}"}

response = requests.post(
    url=f"{API_ROOT}/serving-endpoints/{endpoint_name}/invocations", json=data, headers=headers
)

print(json.dumps(response.json()))